# Open Data Biblioteche di Roma | Download automatico dei csv mensili

![Let's Scrape!](https://media1.giphy.com/media/3o7TKSx0g7RqRniGFG/giphy.gif)

In [1]:
import os
import glob
import subprocess

# Selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.firefox.options import Options
from selenium.common import exceptions
opts = Options()

#facoltativo in ambiente desktop (obbligatorio nel caso di script lato server)
#opts.headless = True 

profile = webdriver.FirefoxProfile()
profile.set_preference("browser.privatebrowsing.autostart", True)
#profile.set_preference("permissions.default.image", 2) #facoltativo

# il driver gecko si scarica qui: https://github.com/mozilla/geckodriver/releases/tag/v0.24.0
# inizializzare il driver specificando come sempre la posizione del geckodriver nel proprio pc/server
driver = (
webdriver.Firefox(
    executable_path='/Users/marcogoldin/Documents/WebDev/Python/Scraping/geckodriver',
    firefox_profile=profile,
    options=opts))

wait_explicit = WebDriverWait(driver, 20)

## Definire la cartella che conterrà le sotto-cartelle con i singoli csv mensili

In [12]:
# es.: '/Users/marco/Documents/OpenDataBibliotecheRoma/csv_prestiti_mensili'
folder_path = ''

## Scraping
### link alla pagina con i csv mensili: https://www.bibliotechediroma.it/opendata?path=DET_P_MESEPREC

In [4]:
driver.get('https://www.bibliotechediroma.it/opendata?path=DET_P_MESEPREC')
wait_explicit.until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="block-bibliotu-content"]/article/ul')))

corpo_article = driver.find_element_by_xpath(
    '//*[@id="block-bibliotu-content"]/article/ul')

blocchi = [
    [i.text.split('\n')] for i in corpo_article.find_elements_by_xpath('//*[@id="block-bibliotu-content"]/article/ul/li')
]

list_all = []
for i in range(len(blocchi)):
    pos = i+1
    data = blocchi[i][0][0]
    files = driver.find_element_by_xpath('//*[@id="block-bibliotu-content"]/article/ul/li'+ str([pos]) + '/ul')
    li = files.find_elements_by_tag_name("a")
    file_list = [elem.get_attribute("href") for elem in li]
    csv = [elem for elem in file_list if elem.endswith('csv')][0]
    list_all.append([data,csv]) # lista contenente la data (anno,mese) e tutti gli url per il download di ogni singolo csv

# Download dei csv, ciascuno in una cartella separata

### - i csv vengono rinominati automaticamente con anno e mese
### - in questo modo sarà più semplice gestire successivamente l'aggregazione dei file o eventuali slice nelle singole sottocartelle...

In [5]:
for i in range(0,len(list_all)):
    if not os.path.exists(os.path.join(folder_path, str(list_all[i][0]))):
        os.makedirs(os.path.join(folder_path, str(list_all[i][0])))
    folder = os.path.join(folder_path, str(list_all[i][0]))
    print(f'cartella: {list_all[i][0]}')
    subprocess.run(['wget','-P',folder,list_all[i][1]])
    os.rename(os.path.join(folder,'DET_P_MESEPREC_01.csv'),os.path.join(folder,list_all[i][0])+'.csv' )

cartella: 201707
cartella: 201708
cartella: 201709
cartella: 201710
cartella: 201711
cartella: 201801
cartella: 201802
cartella: 201803
cartella: 201804
cartella: 201805
cartella: 201806
cartella: 201807
cartella: 201808
cartella: 201809
cartella: 201810
cartella: 201811
cartella: 201812
cartella: 201901
cartella: 201902
cartella: 201903
cartella: 201904
cartella: 201905
cartella: 201906
cartella: 201907


In [6]:
# chiudere il driver Selenium
driver.quit()

In [11]:
csv_files = glob.glob(
    folder_path+'/**/*.csv',
    recursive=True)

print(f'\n*** Lista con elenco di tutti i file ***\n\n {csv_files}')


*** Lista con elenco di tutti i file ***

 ['/Users/marcogoldin/Documents/WebDev/Data_Science/OpenDataBiblioTu/prova_selenium_csv_github/201708/201708.csv', '/Users/marcogoldin/Documents/WebDev/Data_Science/OpenDataBiblioTu/prova_selenium_csv_github/201907/201907.csv', '/Users/marcogoldin/Documents/WebDev/Data_Science/OpenDataBiblioTu/prova_selenium_csv_github/201707/201707.csv', '/Users/marcogoldin/Documents/WebDev/Data_Science/OpenDataBiblioTu/prova_selenium_csv_github/201709/201709.csv', '/Users/marcogoldin/Documents/WebDev/Data_Science/OpenDataBiblioTu/prova_selenium_csv_github/201906/201906.csv', '/Users/marcogoldin/Documents/WebDev/Data_Science/OpenDataBiblioTu/prova_selenium_csv_github/201901/201901.csv', '/Users/marcogoldin/Documents/WebDev/Data_Science/OpenDataBiblioTu/prova_selenium_csv_github/201811/201811.csv', '/Users/marcogoldin/Documents/WebDev/Data_Science/OpenDataBiblioTu/prova_selenium_csv_github/201810/201810.csv', '/Users/marcogoldin/Documents/WebDev/Data_Science/O

## voilà...

![Neato!](https://media3.giphy.com/media/82LD2CCLiR1Epx2xOv/giphy.gif)